In [10]:
################################
# add values based on time coord
# thanks chat
################################

import numpy as np
import pandas as pd
import xarray as xr

# Create a time range
time = pd.date_range('2020-01-01', periods=10, freq='D')

# Create an initial xarray Dataset
ds = xr.Dataset(
    {
        'temperature': (['time'], np.random.rand(len(time)) * 10)  # Random temperature values
    },
    coords={'time': time}
)

# Define a function that depends on time
def temperature_trend(time):
    return 0.5 * (time - np.datetime64('2020-01-01')).astype('timedelta64[D]').astype(int)

# Apply this function to each time point
trend_values = temperature_trend(ds.time.values)

# Add the new values to the dataset
ds['temperature_trend'] = ('time', trend_values)

print(ds)

ds.close()

<xarray.Dataset> Size: 240B
Dimensions:            (time: 10)
Coordinates:
  * time               (time) datetime64[ns] 80B 2020-01-01 ... 2020-01-10
Data variables:
    temperature        (time) float64 80B 6.276 0.2836 1.256 ... 4.115 5.56
    temperature_trend  (time) float64 80B 0.0 0.5 1.0 1.5 ... 3.0 3.5 4.0 4.5


In [11]:
#################################
# add values based on x coord
# should be similar to above code
# thanks chat
#################################

import numpy as np
import pandas as pd
import xarray as xr

# Create a time range
time = pd.date_range('2020-01-01', periods=10, freq='D')
# Create an x range
x = np.linspace(0, 10, 5)

# Create an initial xarray Dataset
ds = xr.Dataset(
    {
        'temperature': (['time', 'x'], np.random.rand(len(time), len(x)) * 10)  # Random temperature values
    },
    coords={'time': time, 'x': x}
)

# Define a function that depends on the x coordinate
def elevation(x):
    return 2 * x + 5

# Apply the function to the x coordinate
elevation_values = elevation(ds.x.values)

# Broadcast the elevation values to match the dimensions of the dataset
elevation_broadcasted = np.broadcast_to(elevation_values, (len(ds.time), len(ds.x)))

# Add the new values to the dataset
ds['elevation'] = (['time', 'x'], elevation_broadcasted)

print(ds)

ds.close()

<xarray.Dataset> Size: 920B
Dimensions:      (time: 10, x: 5)
Coordinates:
  * time         (time) datetime64[ns] 80B 2020-01-01 2020-01-02 ... 2020-01-10
  * x            (x) float64 40B 0.0 2.5 5.0 7.5 10.0
Data variables:
    temperature  (time, x) float64 400B 8.026 7.067 8.173 ... 9.943 1.941 6.951
    elevation    (time, x) float64 400B 5.0 10.0 15.0 20.0 ... 15.0 20.0 25.0


In [39]:
import numpy as np
import pandas as pd
import xarray as xr

vector_file = "May/results/video_capture_2024-05-20_01-34-47_velocimetry_results.nc"
video_file = "May/video_capture_2024-05-20_01-34-47.mp4"

ds = xr.open_dataset(vector_file)

maximum = ds.y.max().item()
minimum = ds.y.min().item()
subdiv = (maximum - minimum) / 5

def spacial_location(y, m, subdiv):
    if y < m + subdiv:
        return 0
    elif (y >= m + subdiv) and (y < m + 2 * subdiv):
        return 1
    elif (y >= m + subdiv) and (y < m + 3 * subdiv):
        return 2
    elif (y >= m + subdiv) and (y < m + 4 * subdiv):
        return 3
    elif (y >= m + subdiv) and (y < m + 5 * subdiv):
        return 4
    else:
        return -1
    # else:
    #     return 4

def spacial_location_wrapper(y_vals):
    maximum = max(y_vals)
    minimum = min(y_vals)
    subdiv = (maximum - minimum) / 5

    print(minimum, subdiv)
    print(y_vals)

    spac_loc = []
    for y in y_vals:
        spac_loc.append(spacial_location(y, minimum, subdiv))
    return spac_loc

spac_loc = spacial_location_wrapper(ds.y.values)

spac_loc

0.15499999999999997 1.8980000000000001
[9.645 9.515 9.385 9.255 9.125 8.995 8.865 8.735 8.605 8.475 8.345 8.215
 8.085 7.955 7.825 7.695 7.565 7.435 7.305 7.175 7.045 6.915 6.785 6.655
 6.525 6.395 6.265 6.135 6.005 5.875 5.745 5.615 5.485 5.355 5.225 5.095
 4.965 4.835 4.705 4.575 4.445 4.315 4.185 4.055 3.925 3.795 3.665 3.535
 3.405 3.275 3.145 3.015 2.885 2.755 2.625 2.495 2.365 2.235 2.105 1.975
 1.845 1.715 1.585 1.455 1.325 1.195 1.065 0.935 0.805 0.675 0.545 0.415
 0.285 0.155]


[-1,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]